# Initial Configs


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Imports

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

#import plotly for visualization
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px
import sys
from IPython.core.display import display, HTML
sys.path.append('..')
pyoff.init_notebook_mode()

from scipy import stats
import swifter

from tqdm import tqdm_notebook as tqdm

from bokeh.resources import INLINE
import bokeh.io
from bokeh import *

# Package configs

In [ ]:
pd.set_option('display.max_columns', None)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('plotting.backend', 'pandas_bokeh')
bokeh.io.output_notebook(INLINE)

# Functions

In [ ]:
# Print rapido para dimensão do Dataframe
def SZ(df):
    print(f"""
--- Dimensão ---
Linhas:  {df.shape[0]}
Colunas: {df.shape[1]}""")

In [ ]:
# Print rapido para dTypes do Dataframe
def DT(df):
    print(f"""
--- DataTypes ---
{df.dtypes}
    """)

# Reading Files

In [ ]:
years = range(2016,2020)

list_datatran = []

for year in years:
    list_datatran.append(pd.read_csv(f"../data/raw/datatran{year}.csv", delimiter=";", encoding='iso-8859-1'))
    
df_datatran = pd.concat(list_datatran)

SZ(df_datatran)
DT(df_datatran)

In [ ]:
# df_pib = pd.read_excel('../data/raw/PIB dos Municípios - base de dados 2010-2017.xls')

In [ ]:
# df_populacao = pd.read_csv('../data/raw/populacao_por_municipio.csv')

In [ ]:
# df_depara_municipio = pd.read_excel('../data/raw/RELATORIO_DTB_BRASIL_MUNICIPIO.xls')

# Data Processing

## PIB Dataset

In [ ]:
# df_pib = df_pib[['Ano', 'Código do Município', 'Nome do Município', 'Código da Unidade da Federação', 'Sigla da Unidade da Federação', 'Nome da Unidade da Federação', 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)', 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)']]
# 
# df_pib.columns = ['ano', 'id_municipio', 'municipio', 'id_uf', 'uf', 'nome_uf', 'pib', 'pib_per_capita']

In [ ]:
# df_pib = df_pib[df_pib['ano'] == 2017]

In [ ]:
# df_pib[['municipio', 'nome_uf']] = df_pib[['municipio', 'nome_uf']].apply(
#     lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.
#     decode('utf-8').str.replace('[^\w\s]', '').str.lower().str.strip())

Criando depara's

In [ ]:
# df_depara_uf = df_pib[['id_uf', 'uf', 'nome_uf']].drop_duplicates()

In [ ]:
# df_depara_municipio = df_pib[['id_municipio', 'municipio']].drop_duplicates()

In [ ]:
# df_depara_uf_municipio = df_pib[['id_municipio', 'municipio', 'id_uf', 'uf', 'nome_uf']].drop_duplicates()

População Dataset

In [ ]:
# df_populacao = df_populacao.merge(df_depara_municipio, how='left', on='id_municipio')

## Datatran Dataset

In [ ]:
df_datatran['data_inversa'] = pd.to_datetime(df_datatran['data_inversa'])

df_datatran[['latitude', 'longitude']] = df_datatran[['latitude', 'longitude']].apply(lambda x: x.str.replace(',','.'))
df_datatran['latitude'] = df_datatran['latitude'].astype('float64')
df_datatran['longitude'] = df_datatran['longitude'].astype('float64')

Normalização de strings

In [ ]:
string_columns = ['dia_semana', 
                  'municipio',
                  'causa_acidente',
                  'tipo_acidente',
                  'classificacao_acidente',
                  'fase_dia',
                  'sentido_via',
                  'condicao_metereologica',
                  'tipo_pista',
                  'tracado_via',
                  'uso_solo',
                  'regional',
                  'delegacia',
                  'uop']

df_datatran[string_columns] = df_datatran[string_columns].apply(
    lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.
    decode('utf-8').str.replace('[^\w\s]', '').str.lower().str.strip())

Processamento da data

In [ ]:
min_data = min(df_datatran['data_inversa'])

df_datatran['d'] = (df_datatran['data_inversa'] - min_data)
df_datatran['d'] / pd.Timedelta(1, unit='d')
df_datatran['d'] = df_datatran['d'].astype('timedelta64[D]')+1

In [ ]:
df_datatran['ano'] = df_datatran['data_inversa'].dt.year

In [ ]:
df_datatran['uso_solo'] = df_datatran.apply(lambda x: 'urbano' if x['uso_solo'] == 'sim' else 'rural', axis=1)

Completar nulos

In [ ]:
df_datatran['km'] = df_datatran['km'].apply(lambda x: str(x).replace(',','.'))
df_datatran = df_datatran[~(df_datatran['br']=='(null)')]
df_datatran['br'] = df_datatran['br'].where(pd.notnull(df_datatran['br']), None)
df_datatran[['br','km']] = df_datatran[['br','km']].fillna(df_datatran[['br','km']].mode().iloc[0])
df_datatran['br'] = df_datatran['br'].astype(int)
df_datatran['km'] = df_datatran['km'].astype(float).fillna(df_datatran['km'].mode().iloc[0]).astype(float).astype(int)

In [ ]:
df_datatran['ufbr'] = df_datatran['uf'] + df_datatran['br'].astype(str)

In [ ]:
df_datatran[['latitude', 'longitude']] = df_datatran[['latitude', 'longitude']].fillna(df_datatran[['latitude', 'longitude']].median())

In [ ]:
df_datatran[string_columns] = df_datatran[string_columns].replace('null', np.nan)
df_datatran[string_columns] = df_datatran[string_columns].fillna(df_datatran[string_columns].mode().iloc[0])

Criar fator de risco (número de acidente na rodovia-km em questão)/(número total de acidentes)

In [ ]:
df_datatran.sort_values(by='d', ascending=True, inplace=True)

In [ ]:
df_datatran['valor_1'] = 1
days_to_analyse = 365

# Criar dataset com qtd de acidentes por br/km num periodo de 1 ano ate a data do acidente em questao
df_acidentes_brkm = df_datatran.groupby(['br', 'km']).rolling(f'{days_to_analyse}D', on="data_inversa")['valor_1'].sum().reset_index(name='qtd_acidentes_brkm')
df_acidentes_brkm.drop_duplicates(subset=['br', 'km', 'data_inversa'], keep='last', inplace=True)

# Criar dataset com qtd total de acidentes num periodo de 1 ano ate a data do acidente em questao
df_acidentes_brasil = df_datatran.groupby(['valor_1']).rolling(f'{days_to_analyse}D', on="data_inversa")['valor_1'].sum().reset_index(name='qtd_acidentes_brasil')
df_acidentes_brasil = df_acidentes_brasil.drop_duplicates(subset='data_inversa', keep='last')[['data_inversa', 'qtd_acidentes_brasil']]

# Join datasets
df_datatran = df_datatran.merge(df_acidentes_brkm, how='left', on=['br', 'km', 'data_inversa'])
df_datatran = df_datatran.merge(df_acidentes_brasil, how='left', on=['data_inversa'])

# Criar atributo risco
df_datatran['risco'] = df_datatran['qtd_acidentes_brkm']/df_datatran['qtd_acidentes_brasil']

# Remover colunas indesejadas
del df_datatran['qtd_acidentes_brkm']
del df_datatran['qtd_acidentes_brasil']
del df_datatran['valor_1']

# Remover dados de acidentes do primeiro ano de analise por nao haver dados do ano antecedente para criar o atributo risco (lembrar que a analise é feita em um periodo de 1 ano)
df_datatran = df_datatran[df_datatran['data_inversa'].dt.year >= years[1]]

In [ ]:
# df_datatran_mortes = df_datatran[df_datatran['mortos']!=0]
# df_datatran['contem_vitima_fatal'] = df_datatran.apply(lambda x: 1 if x['mortos'] != 0 else 0, axis=1)
# 
# # Criar dataset com qtd de acidentes por br/km num periodo de 1 ano ate a data do acidente em questao
# df_acidentes_brkm = df_datatran_mortes.groupby(['br', 'km']).rolling(f'{days_to_analyse}D', on="data_inversa")['contem_vitima_fatal'].sum().reset_index(name='qtd_acidentes_brkm')
# df_acidentes_brkm.drop_duplicates(subset=['br', 'km', 'data_inversa'], keep='last', inplace=True)
# 
# # Criar dataset com qtd total de acidentes num periodo de 1 ano ate a data do acidente em questao
# df_acidentes_brasil = df_datatran_mortes.groupby(['contem_vitima_fatal']).rolling(f'{days_to_analyse}D', on="data_inversa")['contem_vitima_fatal'].sum().reset_index(name='qtd_acidentes_brasil')
# df_acidentes_brasil = df_acidentes_brasil.drop_duplicates(subset='data_inversa', keep='last')[['data_inversa', 'qtd_acidentes_brasil']]
# 
# # Join datasets
# df_datatran_mortes = df_datatran_mortes.merge(df_acidentes_brkm, how='left', on=['br', 'km', 'data_inversa'])
# df_datatran_mortes = df_datatran_mortes.merge(df_acidentes_brasil, how='left', on=['data_inversa'])
# 
# # Criar atributo risco
# df_datatran_mortes['risco'] = df_datatran_mortes['qtd_acidentes_brkm']/df_datatran_mortes['qtd_acidentes_brasil']
# 
# # Remover colunas indesejadas
# del df_datatran_mortes['qtd_acidentes_brkm']
# del df_datatran_mortes['qtd_acidentes_brasil']

In [ ]:
plot_data = [
    go.Histogram(
        x=df_datatran['risco']
    )
]

plot_layout = go.Layout(
        title='Distribuição do risco'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
plot_data = [
    go.Histogram(
        x=np.log(df_datatran['risco'])
    )
]

plot_layout = go.Layout(
        title='Distribuição do risco'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
# df_datatran['risco'] = df_datatran.groupby(['br', 'km'])['valor_1'].cumsum()
# df_acidentes_ate_data = df_datatran.groupby(['data_inversa', 'valor_1'])['valor_1'].sum()
# df_acidentes_ate_data = df_acidentes_ate_data.groupby(['valor_1']).cumsum().reset_index(name='qtd_acidentes_ate_data')
# df_datatran = df_datatran.merge(df_acidentes_ate_data[['data_inversa', 'qtd_acidentes_ate_data']], how='left', on='data_inversa')

Transformar dia da semana em dado categórico numérico

In [ ]:
def weekday_process(df, weekday_column):
    return np.select(
        [
            df[weekday_column].str.contains('seg'),
            df[weekday_column].str.contains('ter'),
            df[weekday_column].str.contains('qua'),
            df[weekday_column].str.contains('qui'),
            df[weekday_column].str.contains('sex'),
            df[weekday_column].str.contains('sab'),
            df[weekday_column].str.contains('dom')
        ],
        [
            'seg', 'ter', 'qua', 'qui', 'sex', 'sab', 'dom'
        ],
        ''
    )

In [ ]:
df_datatran['dia_semana'] = weekday_process(df_datatran, weekday_column='dia_semana')

Agrupar causas de acidentes para diminuir a cardinalidade do problema

In [ ]:
# dict_causa_acidente = {
#     'falta de atencao':'falta de atencao', 
#     'falta de atencao a conducao':'falta de atencao',
#     'dormindo':'dormindo', 
#     'condutor dormindo':'dormindo',
#     'deficiencia ou nao acionamento do sistema de iluminacaosinalizacao do veiculo': 'defeito mecanico', 
#     'defeito mecanico em veiculo':'defeito mecanico', 
#     'avarias eou desgaste excessivo no pneu':'defeito mecanico', 
#     'defeito mecanico no veiculo':'defeito mecanico',
#     'ingestao de alcool':'ingestao de substancias psicoativas', 
#     'ingestao de substancias psicoativas':'ingestao de substancias psicoativas',
#     'restricao de visibilidade':'restricao de visibilidade', 
#     'sinalizacao da via insuficiente ou inadequada':'restricao de visibilidade',
#     'outras':'outras', 
#     'animais na pista':'animais na pista', 
#     'desobediencia a sinalizacao':'desobediencia a sinalizacao', 
#     'velocidade incompativel':'velocidade incompativel', 
#     'ultrapassagem indevida':'ultrapassagem indevida', 
#     'nao guardar distancia de seguranca':'nao guardar distancia de seguranca', 
#     'defeito na via':'defeito na via', 
#     'fenomenos da natureza':'fenomenos da natureza', 
#     'desobediencia as normas de transito pelo condutor':'desobediencia as normas de transito pelo condutor', 
#     'pista escorregadia':'pista escorregadia', 
#     'mal subito':'mal subito', 
#     'carga excessiva eou mal acondicionada':'carga excessiva eou mal acondicionada', 
#     'objeto estatico sobre o leito carrocavel':'objeto estatico sobre o leito carrocavel', 
#     'agressao externa':'agressao externa', 
#     'falta de atencao do pedestre':'falta de atencao do pedestre', 
#     'desobediencia as normas de transito pelo pedestre':'desobediencia as normas de transito pelo pedestre', 
#     'ingestao de alcool eou substancias psicoativas pelo pedestre':'ingestao de alcool eou substancias psicoativas pelo pedestre'
# }

In [ ]:
# df_datatran['causa_acidente'] = df_datatran.apply(lambda x: dict_causa_acidente[x['causa_acidente']], axis=1)

Agrupar tipo de acidentes para diminuir a cardinalidade do problema

In [ ]:
# dict_tipo_acidente = {
#     'colisao traseira': 'colisao',
#     'saida de pista': 'capotamento',
#     'colisao com objeto fixo': 'colisao',
#     'colisao transversal': 'colisao',
#     'tombamento': 'capotamento',
#     'queda de motocicleta  bicicleta  veiculo': 'capotamento',
#     'atropelamento de animal': 'atropelamento', 
#     'capotamento': 'capotamento',
#     'colisao lateral': 'colisao',
#     'colisao frontal': 'colisao',
#     'atropelamento de pessoa': 'atropelamento',
#     'colisao com objeto movel': 'colisao',
#     'danos eventuais': 'outras',
#     'incendio': 'outras',
#     'colisao com bicicleta': 'colisao',
#     'derramamento de carga': 'outras',
#     'queda de ocupante de veiculo': 'atropelamento',
#     'colisao com objeto estatico': 'colisao',
#     'saida de leito carrocavel': 'outras',
#     'atropelamento de pedestre': 'atropelamento',
#     'colisao com objeto em movimento': 'colisao',
#     'engavetamento': 'colisao'
# }

In [ ]:
# df_datatran['tipo_acidente'] = df_datatran.apply(lambda x: dict_tipo_acidente[x['tipo_acidente']], axis=1)

Transformar coordenadas (latitude, longitude) em espaço cartesiano

In [ ]:
df_datatran['coordenada_x'] = np.cos(df_datatran['latitude']) * np.cos(df_datatran['longitude'])
df_datatran['coordenada_y'] = np.cos(df_datatran['latitude']) * np.sin(df_datatran['longitude'])
df_datatran['coordenada_z'] = np.sin(df_datatran['latitude'])

### Feriados

Criar atributo que indica distância em dias entre data do acidente e data de feriados

In [ ]:
df_holidays = pd.read_parquet('../data/raw/holidays.parquet')

In [ ]:
# Considerar feriados apenas entre a data de analise
data_inicio_analise = f'{years[1]}-01-01'
data_fim_analise = f'{years[-1]}-12-31'

df_holidays["data"] = pd.to_datetime(df_holidays["data"], format='%d/%m/%Y')
df_holidays = df_holidays[(df_holidays['data'] >= data_inicio_analise) & (df_holidays['data'] <= data_fim_analise)]
df_holidays = df_holidays.add_prefix('holiday_')
df_holidays

In [ ]:
df_holidays.fillna('', inplace=True)
df_holidays[['holiday_municipio']] = df_holidays[['holiday_municipio']].apply(
    lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.
    decode('utf-8').str.replace('[^\w\s]', '').str.lower().str.strip())

In [ ]:
days_offset_holiday = 2
df_holidays[f'holiday_date_minus_{days_offset_holiday}'] = df_holidays['holiday_data'] + pd.DateOffset(-days_offset_holiday)
df_holidays[f'holiday_date_plus_{days_offset_holiday}'] = df_holidays['holiday_data'] + pd.DateOffset(days_offset_holiday)

In [ ]:
df_holidays = df_holidays[(df_holidays['holiday_tipo'] == 'ESTADUAL') |
                          (df_holidays['holiday_tipo'] == 'NACIONAL') |
                          (df_holidays['holiday_nome'] == 'Carnaval')]

In [ ]:
import pandasql as ps

sqlcode = f'''
select *
from df_datatran
inner join df_holidays on 
    df_datatran.data_inversa >= df_holidays.holiday_date_minus_{days_offset_holiday}
    AND df_datatran.data_inversa <= df_holidays.holiday_date_plus_{days_offset_holiday}
WHERE
    -- (df_holidays.holiday_tipo LIKE 'MUNICIPAL' AND df_datatran.municipio = df_holidays.holiday_municipio)
    (df_holidays.holiday_tipo LIKE 'ESTADUAL' AND df_datatran.uf = df_holidays.holiday_uf)
    OR df_holidays.holiday_tipo LIKE 'NACIONAL'
    OR df_holidays.holiday_nome LIKE 'Carnaval'
'''

df_datatran_near_holidays = ps.sqldf(sqlcode,locals())

In [ ]:
df_datatran_near_holidays["data_inversa"] = pd.to_datetime(df_datatran_near_holidays["data_inversa"])
df_datatran_near_holidays["holiday_data"] = pd.to_datetime(df_datatran_near_holidays["holiday_data"])
df_datatran_near_holidays['diff_ac_feriado_dias'] = df_datatran_near_holidays['data_inversa'] - df_datatran_near_holidays['holiday_data']
df_datatran_near_holidays['diff_ac_feriado_dias'] = df_datatran_near_holidays['diff_ac_feriado_dias'].astype('timedelta64[D]').astype(int)

In [ ]:
df_datatran_near_holidays['diff_ac_feriado_dias'] = df_datatran_near_holidays['diff_ac_feriado_dias'].astype('category')
df_datatran_near_holidays['diff_ac_feriado_dias'] = pd.Categorical(df_datatran_near_holidays['diff_ac_feriado_dias'], categories=[0,-1,1,-2,2, np.inf], ordered=True)
df_datatran_near_holidays = df_datatran_near_holidays.sort_values('diff_ac_feriado_dias').groupby('id', as_index=False).first()
df_datatran_near_holidays['em_janela_feriado'] = 1

In [ ]:
# df_datatran = df_datatran.merge(df_datatran_near_holidays[['id', 'diff_ac_feriado_dias', 'holiday_data', 'holiday_nome']], how='left', on=['id'])
df_datatran = df_datatran.merge(df_datatran_near_holidays[['id', 'em_janela_feriado']], how='left', on=['id'])
df_datatran['em_janela_feriado'] = df_datatran['em_janela_feriado'].fillna(0)
df_datatran.drop_duplicates(inplace=True)

In [ ]:
df_datatran.sort_values(["data_inversa"], ascending=True, inplace=True)

In [ ]:
# df_datatran['dia'] = df_datatran['data_inversa'].dt.day
# df_datatran['mes'] = df_datatran['data_inversa'].dt.month
# df_datatran['ano'] = df_datatran['data_inversa'].dt.year
# df_datatran['diadoano'] = df_datatran['data_inversa'].dt.dayofyear

In [ ]:
# plot_data = [
#     go.Histogram(
#         x=df_datatran['diff_ac_feriado_dias']
#     )
# ]
# 
# plot_layout = go.Layout(
#         title='Número acidentes por janela feriado'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

In [ ]:
# plot_data = [
#     go.Histogram(
#         x=df_datatran['holiday_nome']
#     )
# ]
# 
# plot_layout = go.Layout(
#         title='Número acidentes por feriado'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

Tranformar target em variável binária

In [ ]:
# df_datatran['contem_vitima_fatal'] = df_datatran.apply(lambda x: 1 if x['mortos'] != 0 else 0, axis=1)

In [ ]:
# Class count
count_class_0, count_class_1 = df_datatran['contem_vitima_fatal'].value_counts()

# Divide by class
df_class_0 = df_datatran[df_datatran['contem_vitima_fatal'] == 0]
df_class_1 = df_datatran[df_datatran['contem_vitima_fatal'] == 1]

Random Under-Sampling

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under['contem_vitima_fatal'].value_counts())

df_test_under['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Count (target)');

Random Over-Sampling

In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over['contem_vitima_fatal'].value_counts())

df_test_over['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
df_test_over.to_parquet('../data/processed/dataset_v3.parquet', index=False)

In [ ]:
# df_test_over.to_parquet('../data/processed/dataset_over_v2.parquet', index=False)

In [ ]:
# df_test_under.to_parquet('../data/processed/dataset_under_v2.parquet', index=False)

## Join Datasets

In [ ]:
df_final = df_datatran.merge(df_pib[['municipio', 'uf', 'id_municipio', 'pib', 'pib_per_capita']], how='left', on=['uf', 'municipio'])

In [ ]:
df_final = df_final.merge(df_populacao, how='left', on=['ano', 'id_municipio', 'municipio'])

In [ ]:
df_final[['pib', 'pib_per_capita', 'populacao']] = df_final[['uf', 'pib', 'pib_per_capita', 'populacao']].groupby("uf").transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_final.head()

In [ ]:
# df_final.to_csv('../data/processed/dataset_v4.csv', index=False, encoding='iso-8859-1')
# df_final.to_parquet('../data/processed/dataset_v4.parquet', index=False)

# Exploratory Analysis

In [ ]:
df_datatran[['pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos', 'ignorados', 'feridos', 'veiculos']].describe()

In [ ]:
df_datatran.head()

In [ ]:
df_datatran_causa_acidente_count = df_datatran.groupby(['causa_acidente'])['id'].count().reset_index()

df_datatran_causa_acidente_count.set_index('causa_acidente', inplace=True)

In [ ]:
df_datatran_causa_acidente_count.sort_values(by='id', ascending=True).plot(kind='barh', figsize=(10,10))

# K-Means

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# 
# lb_make = LabelEncoder()
# tx_data["SEGMENTO_AZURE_CODE"] = lb_make.fit_transform(tx_data["SEGMENTO_AZURE"])

In [ ]:
drop_columns = [
    'id'
    , 'br'
    , 'km'
    , 'classificacao_acidente'
    , 'ano'
    , 'data_inversa'
    , 'horario'
    , 'mortos'
    , 'regional'
    , 'delegacia'
    , 'uop'
    , 'municipio'
    , 'pessoas'
    , 'feridos_leves'
    , 'feridos_graves'
    , 'ilesos'
    , 'ignorados'
    , 'feridos'
    , 'veiculos'
]

In [ ]:
z_score = stats.zscore(df_datatran.drop(drop_columns, axis=1).drop(['contem_vitima_fatal'], axis=1).select_dtypes(include=[np.number]))
abs_z_scores = np.abs(z_score)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_df = df_datatran[filtered_entries]
df_kmeans = new_df.drop(drop_columns, axis=1)

In [ ]:
plot_data = [
    go.Histogram(
        x = df_kmeans['contem_vitima_fatal'],
        orientation='v',
        name='Acidentes Vítima Fatal'
    )
]

plot_layout = go.Layout(
        width=700,
        height=300
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
category_cols = [
      'dia_semana'
    , 'uf'
#     , 'br'
#     , 'km'
#     , 'municipio'
    , 'causa_acidente'
    , 'tipo_acidente'
#     , 'classificacao_acidente'
    , 'fase_dia'
    , 'sentido_via'
    , 'condicao_metereologica'
    , 'tipo_pista'
    , 'tracado_via'
    , 'uso_solo'
]

df_kmeans = df_kmeans.drop(category_cols, axis=1).reset_index(drop=True).merge(pd.get_dummies(df_kmeans[category_cols]).reset_index(drop=True), how='left', left_index=True, right_index=True)

In [ ]:
del df_kmeans['contem_vitima_fatal']

In [ ]:
from sklearn.cluster import KMeans

sse={}

with tqdm(total = len(range(1, 10))) as pbar:
    for k in range(1, 10):
        kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df_kmeans)
        df_kmeans["clusters"] = kmeans.labels_
        sse[k] = kmeans.inertia_ 
        
        pbar.update()
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.show()

In [ ]:
#build 4 clusters for recency and add it to dataframe
kmeans = KMeans(n_clusters=5)
kmeans.fit(df_kmeans)
df_kmeans['Cluster K-Means'] = kmeans.predict(df_kmeans)

In [ ]:
plot_data = [
    go.Histogram(
        x = df_kmeans['Cluster K-Means'],
        orientation='v',
        name='Clusters'
    )
]

plot_layout = go.Layout(
        width=700,
        height=300
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
df_datatran